In [1]:
import numpy
from sklearn.metrics import accuracy_score
import tensorflow as tf
import matplotlib.pyplot as plt

from tslearn.datasets import CachedDatasets
from tslearn.preprocessing import TimeSeriesScalerMinMax
from tslearn.shapelets import LearningShapelets, \
    grabocka_params_to_shapelet_size_dict
from tslearn.utils import ts_size
import numpy
from sklearn.metrics import accuracy_score

from tslearn.generators import random_walk_blobs
from tslearn.preprocessing import TimeSeriesScalerMinMax, \
    TimeSeriesScalerMeanVariance
from tslearn.neighbors import KNeighborsTimeSeriesClassifier, \
    KNeighborsTimeSeries
import pandas as pd
import numpy as np
import random
from tslearn.datasets import UCR_UEA_datasets
import sktime
from tslearn.preprocessing import TimeSeriesResampler
import random
from sktime.classification.kernel_based import RocketClassifier
from sktime.datasets import load_unit_test

In [56]:
from sktime.classification.kernel_based import RocketClassifier
from sktime.datasets import load_unit_test
X_train, y_train = load_unit_test(split="train", return_X_y=True)
X_test, y_test = load_unit_test(split="test", return_X_y=True) 
clf = RocketClassifier(num_kernels=500) 
clf.fit(X_train, y_train) 
y_pred = clf.predict(X_test) 

In [57]:
df = pd.read_pickle("Train.pkl")
randomlist1 = random.sample(range(0, 260), 78)

df_test1 = df.loc[df["Batch"].isin(randomlist1)]
df_train1 = df.loc[~df["Batch"].isin(randomlist1)]
df_steps1 = df_test1.loc[df_test1["CuStepNo ValueY"] != df_test1["Next_Step"]]
df_steps2 = df_train1.loc[df_train1["CuStepNo ValueY"] != df_train1["Next_Step"]]

In [58]:
y_train1 = df_steps2.loc[df_steps2["CuStepNo ValueY"] == 1,"DeviationID ValueY"].astype(int)
y_test1 = df_steps1.loc[df_steps1["CuStepNo ValueY"] == 1,"DeviationID ValueY"].astype(int)
X_train1 = df_train1.drop(["DeviationID ValueY","Next_Step","timestamp","Unnamed: 0"],axis=1)
X_test1 = df_test1.drop(["DeviationID ValueY","Next_Step","timestamp","Unnamed: 0"],axis=1)

In [59]:
X_train_step1 = []
for i in X_train1.Batch.unique():
        num = numpy.array(X_train1.loc[(X_train1["Batch"] == i)&(X_train1["CuStepNo ValueY"] == 1)].drop(["Batch","CuStepNo ValueY"],axis=1))
        X_train_step1.append(num)

In [60]:
X_test_step1 = []
for i in X_test1.Batch.unique():
        num = numpy.array(X_test1.loc[(X_test1["Batch"] == i)&(X_test1["CuStepNo ValueY"] == 1)].drop(["Batch","CuStepNo ValueY"],axis=1))
        X_test_step1.append(num)

In [62]:
length = []
for i in X_train_step1:
    length.append(len(i))

In [64]:
X_train_s1 = TimeSeriesResampler(sz=int(np.median(length))).fit_transform(X_train_step1)
X_test_s1 = TimeSeriesResampler(sz=int(np.median(length))).fit_transform(X_test_step1)

In [54]:
y_train1

2102      5
3212      1
4263      6
7334      9
8449      9
         ..
260433    9
261541    9
262655    9
263774    9
264886    9
Name: DeviationID ValueY, Length: 174, dtype: int32

In [68]:
X_test_s1.shape[1]

248

In [69]:
X_train = X_train_s1.reshape(X_train_s1.shape[0],39,X_train_s1.shape[1])

In [70]:
X_test = X_test_s1.reshape(X_test_s1.shape[0],39,X_test_s1.shape[1])

In [37]:
scaler = TimeSeriesScalerMinMax()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
scaler = TimeSeriesScalerMinMax()
X_train = scaler.fit_transform(X_train_s1)
X_test = scaler.transform(X_test_s1)

In [71]:
clf = RocketClassifier(num_kernels=500) 
clf.fit(X_train, y_train1) 
y_pred = clf.predict(X_test) 

In [45]:
clf.estimator_

SklearnClassifierPipeline(classifier=RidgeClassifierCV(alphas=array([1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
       4.64158883e-01, 2.15443469e+00, 1.00000000e+01, 4.64158883e+01,
       2.15443469e+02, 1.00000000e+03])),
                          transformers=[Rocket(num_kernels=500),
                                        TabularToSeriesAdaptor(transformer=StandardScaler(with_mean=False))])

In [72]:
accuracy_score(y_test1, y_pred)

0.7466666666666667

In [73]:
y_pred

array([ 3,  4,  9,  1,  3,  1,  1,  1,  1,  1,  1,  5,  1,  1,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  3,  3,  3,  3,  3,  3,  3,  3,  4,  4,  4,
        4,  4,  1,  1,  1,  5,  1,  1,  1,  1,  6,  7,  6,  7,  7, 10,  7,
        1,  7,  6,  7, 10,  9,  8,  8,  8,  8,  9,  9,  9,  9,  9,  9,  9,
        9,  9,  6,  6, 10, 10, 10])

In [75]:
y_train2 = df_steps2.loc[df_steps2["CuStepNo ValueY"] == 7,"DeviationID ValueY"].astype(int)
y_test2 = df_steps1.loc[df_steps1["CuStepNo ValueY"] == 7,"DeviationID ValueY"].astype(int)
X_train1 = df_train1.drop(["DeviationID ValueY","Next_Step","timestamp","Unnamed: 0"],axis=1)
X_test1 = df_test1.drop(["DeviationID ValueY","Next_Step","timestamp","Unnamed: 0"],axis=1)

In [74]:
X_train_step7 = []
for i in X_train1.Batch.unique():
        num = numpy.array(X_train1.loc[(X_train1["Batch"] == i)&(X_train1["CuStepNo ValueY"] == 7)].drop(["Batch","CuStepNo ValueY"],axis=1))
        X_train_step7.append(num)
X_test_step7 = []
for i in X_test1.Batch.unique():
        num = numpy.array(X_test1.loc[(X_test1["Batch"] == i)&(X_test1["CuStepNo ValueY"] == 7)].drop(["Batch","CuStepNo ValueY"],axis=1))
        X_test_step7.append(num)

In [76]:
length = []
for i in X_train_step7:
    length.append(len(i))
X_train_s7 = TimeSeriesResampler(sz=int(np.median(length))).fit_transform(X_train_step7)
X_test_s7 = TimeSeriesResampler(sz=int(np.median(length))).fit_transform(X_test_step7)

In [77]:
X_train_7 = X_train_s7.reshape(X_train_s7.shape[0],39,X_train_s7.shape[1])
X_test_7 = X_test_s7.reshape(X_test_s7.shape[0],39,X_test_s7.shape[1])

In [79]:
clf = RocketClassifier(num_kernels=500) 
clf.fit(X_train_7, y_train2) 
y_pred = clf.predict(X_test_7) 
accuracy_score(y_test2, y_pred)

0.88

In [80]:
y_train8 = df_steps2.loc[df_steps2["CuStepNo ValueY"] == 8,"DeviationID ValueY"].astype(int)
y_test8 = df_steps1.loc[df_steps1["CuStepNo ValueY"] == 8,"DeviationID ValueY"].astype(int)
X_train1 = df_train1.drop(["DeviationID ValueY","Next_Step","timestamp","Unnamed: 0"],axis=1)
X_test1 = df_test1.drop(["DeviationID ValueY","Next_Step","timestamp","Unnamed: 0"],axis=1)
X_train_step8 = []
for i in X_train1.Batch.unique():
        num = numpy.array(X_train1.loc[(X_train1["Batch"] == i)&(X_train1["CuStepNo ValueY"] == 8)].drop(["Batch","CuStepNo ValueY"],axis=1))
        X_train_step8.append(num)
X_test_step8 = []
for i in X_test1.Batch.unique():
        num = numpy.array(X_test1.loc[(X_test1["Batch"] == i)&(X_test1["CuStepNo ValueY"] == 8)].drop(["Batch","CuStepNo ValueY"],axis=1))
        X_test_step8.append(num)

In [81]:
length = []
for i in X_train_step8:
    length.append(len(i))
X_train_s8 = TimeSeriesResampler(sz=int(np.median(length))).fit_transform(X_train_step8)
X_test_s8 = TimeSeriesResampler(sz=int(np.median(length))).fit_transform(X_test_step8)

In [83]:
X_train_8 = X_train_s8.reshape(X_train_s8.shape[0],39,X_train_s8.shape[1])
X_test_8 = X_test_s8.reshape(X_test_s8.shape[0],39,X_test_s8.shape[1])

In [84]:
clf = RocketClassifier(num_kernels=500) 
clf.fit(X_train_8, y_train8) 
y_pred = clf.predict(X_test_8) 
accuracy_score(y_test8, y_pred)

0.92

In [85]:
y_train3 = df_steps2.loc[df_steps2["CuStepNo ValueY"] == 3,"DeviationID ValueY"].astype(int)
y_test3 = df_steps1.loc[df_steps1["CuStepNo ValueY"] == 3,"DeviationID ValueY"].astype(int)
X_train1 = df_train1.drop(["DeviationID ValueY","Next_Step","timestamp","Unnamed: 0"],axis=1)
X_test1 = df_test1.drop(["DeviationID ValueY","Next_Step","timestamp","Unnamed: 0"],axis=1)
X_train_step3 = []
for i in X_train1.Batch.unique():
        num = numpy.array(X_train1.loc[(X_train1["Batch"] == i)&(X_train1["CuStepNo ValueY"] == 3)].drop(["Batch","CuStepNo ValueY"],axis=1))
        X_train_step3.append(num)
X_test_step3 = []
for i in X_test1.Batch.unique():
        num = numpy.array(X_test1.loc[(X_test1["Batch"] == i)&(X_test1["CuStepNo ValueY"] == 3)].drop(["Batch","CuStepNo ValueY"],axis=1))
        X_test_step3.append(num)

In [86]:
length = []
for i in X_train_step8:
    length.append(len(i))
X_train_s3 = TimeSeriesResampler(sz=int(np.median(length))).fit_transform(X_train_step3)
X_test_s3 = TimeSeriesResampler(sz=int(np.median(length))).fit_transform(X_test_step3)

In [87]:
X_train_3 = X_train_s3.reshape(X_train_s3.shape[0],39,X_train_s3.shape[1])
X_test_3 = X_test_s3.reshape(X_test_s3.shape[0],39,X_test_s3.shape[1])

In [88]:
clf = RocketClassifier(num_kernels=500) 
clf.fit(X_train_8, y_train8) 
y_pred = clf.predict(X_test_8) 
accuracy_score(y_test8, y_pred)

0.92